# Метод Монте-Карло

In [1]:
# !pip install tqdm

In [2]:
import numpy as np
from tqdm.notebook import tqdm

In [3]:
N = 1000000
dim = 10
is_inside = []
for _ in tqdm(range(N)):
    x = 2 * np.random.rand(dim) - 1
    is_inside.append(sum(x ** 2) <= 1)
print(f'{"Monte Carlo:":15}{2 ** dim * np.mean(is_inside):.3f} +/- {2 ** dim * np.std(is_inside) / np.sqrt(N):.3f}')

from scipy.special import gamma

print(f'{"True:":15}{np.pi ** (dim / 2) / gamma(dim / 2 + 1):.3f}')

print(f'{"Dots inside:":15}{100 * np.mean(is_inside):.4f}%')


Monte Carlo:   2.500 +/- 0.051
True:          2.550
Dots inside:   0.2441%


In [4]:
N = 1000000
batch = 100000
dim = 10
is_inside = 0
for _ in tqdm(range(N // batch)):
    x = 2 * np.random.rand(dim, batch) - 1
    is_inside += ((x ** 2).sum(axis=0) <= 1).sum()
mean = is_inside / N
print(f'{"Monte Carlo:":15}{2 ** dim * mean:.3f} +/- {2 ** dim * np.sqrt(mean * (1 - mean)) / np.sqrt(N):.3f}')
print(f'{"True:":15}{np.pi ** (dim / 2) / gamma(dim / 2 + 1):.3f}')
print(f'{"Dots inside:":15}{mean:.4f}%')


Monte Carlo:   2.526 +/- 0.051
True:          2.550
Dots inside:   0.0025%


# 21

In [5]:
class Player:
    def __init__(self):
        self.hand = []
        self.points = [0]
        self.turn = 0
        
    def new_game(self, turn):
        self.hand = []
        self.points = [0]
        self.turn = turn
        
    def wanna_more(self, info):
        return False
    
    def give(self, card):
        self.points = [p + card for p in self.points]
        if card == 11:
            self.points.extend([p - 10 for p in self.points])

class DummyPlayer(Player):
    
    def __init__(self, thres):
        super().__init__()
        self.thres = thres
    
    def wanna_more(self, info):
        points = max([p if p <= 21 else 0 for p in self.points])
        return points < self.thres and min(self.points) <= 21


def twenty_one(players):
    deck = [6, 7, 8, 9, 10, 2, 3, 4, 11] * 4
    np.random.shuffle(deck)
    for i, name in enumerate(players):
        players[name].new_game(i)
    hands = {name: [] for name in players}
    takes = {name: True for name in players}
    points = {name: [0] for name in players}
    while deck and any(takes.values()):
        for name in players:
            if takes[name]:
                if players[name].wanna_more(takes):
                    card = deck.pop(0)
                    players[name].give(card)
                    hands[name].append(card)
                    points[name] = [p + card for p in points[name]]
                    if card == 11:
                        points[name].extend([p - 10 for p in points[name]])
                    if not deck:
                        break
                else:
                    takes[name] = False
    for name in players:
        points[name] = max([p if p <= 21 else 0 for p in points[name]])
    return points

In [6]:
players = {'A': DummyPlayer(18), 'B': DummyPlayer(19)}
wins = {'A': 0, 'B': 0}

for i in tqdm(range(100000)):
    points = twenty_one(players)
    if points['A'] > points['B']:
        wins['A'] += 1
    if points['A'] < points['B']:
        wins['B'] += 1
        
print(wins)
N = (wins['A'] + wins['B'])
winrate = wins['A'] / N
std = np.sqrt(winrate * (1 - winrate))
print(f'Winrate A: {100 * (winrate - 2 * std / np.sqrt(N)):.2f}-{100 * (winrate + 2 * std / np.sqrt(N)):.2f}%')


{'A': 38851, 'B': 37611}
Winrate A: 50.45-51.17%


# Решение ДЗ №1.

Для начала нужно создать данные для обучения: 
 * Автомат с вероятностями перехода,
 * Игру для сбора данных против Dummy(18).

In [7]:
states = [(cards, points) for cards in np.arange(13) for points in np.arange(22)] 
#12 карт гарантированный 0: 1 * 4 + 2 * 4 + 3 * 4 = 24 --- минимально возможная сумма
#очков в игре от 0 до 21

machine = {x: {y: 0 for y in states} for x in states} #Автомат



Теперь сделаем парня который берет ровно n карт:

В основном он такой же как и `DummyPlayer`, но чтобоы не реализовывать игру лишний раз заново, хочется записывать данные о состояниях для машины переходов с помощью функции `give()`. Также создадим метод `getpoints()`, который делает понятно что и который давно пора добавить:

In [9]:
class FinalCountdown(Player):
    
    def __init__(self, countDown):
        super().__init__()
        self.countDown = countDown
        
    def getPoints(self):
        return max([t if t <= 21 else 0 for t in self.points()])
    
    def wanna_more(self, info):
        return len(hand) < countDown
    
    def give(self, card):
        curopt = getPoints()
        super().give()
        states[(len(hand), curopt)][(len(hand) + 1, getPoints())] += 1
